In [ ]:
!pip install git+https://github.com/afnan47/cuda.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/afnan47/cuda.git to /tmp/pip-req-build-5rnxxx3c
  Running command git clone --filter=blob:none --quiet https://github.com/afnan47/cuda.git /tmp/pip-req-build-5rnxxx3c
  Resolved https://github.com/afnan47/cuda.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4287 sha256=22b3c54141e579f9a3ab454a554307dab83e499146aa9698fa78a2c8ad3c6227
  Stored in directory: /tmp/pip-ephem-wheel-cache-6erdme4y/wheels/aa/f3/44/e10c1d226ec561d971fcd4b0463f6bff08602afa928a3e7bc7
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu
#include <iostream>
#include <stdio.h>
#include <stdlib.h>

#define SIZE 16

__global__
void matrixMul(int* dev_a, int* dev_b, int* dev_c)
{
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
 
    if (row < SIZE && col < SIZE)
    {
        int sum = 0;
     
        for (int i = 0; i < SIZE; ++i)
        {
            sum += dev_a[row * SIZE + i] * dev_b[i * SIZE + col];
        }
        dev_c[row * SIZE + col] = sum;
    }
}

int main()
{
    int* a, *b, *c;
 
    int matrix_size = SIZE * SIZE * sizeof(int);
    a = (int*)malloc(matrix_size);
    b = (int*)malloc(matrix_size);
    c = (int*)malloc(matrix_size);
 
    int* dev_a, *dev_b, *dev_c;
    cudaMalloc(&dev_a, matrix_size);
    cudaMalloc(&dev_b, matrix_size);
    cudaMalloc(&dev_c, matrix_size);
 
    for (int i = 0; i < SIZE * SIZE; ++i)
    {
        a[i] = rand() % 10;
        b[i] = rand() % 10;
    }
 
    cudaMemcpy(dev_a, a, matrix_size, cudaMemcpyHostToDevice);
    cudaMemcpy(dev_b, b, matrix_size, cudaMemcpyHostToDevice);
 
    dim3 blockDim(SIZE, SIZE);
    dim3 gridDim((SIZE + blockDim.x - 1) / blockDim.x, (SIZE + blockDim.y - 1) / blockDim.y);
 
    matrixMul<<<gridDim, blockDim>>>(dev_a, dev_b, dev_c);
 
    cudaMemcpy(c, dev_c, matrix_size, cudaMemcpyDeviceToHost);
 
    // Print result
    printf("Matrix C:\n");
    for (int i = 0; i < SIZE; i++) {
        for (int j = 0; j < SIZE; j++) {
            printf("%d ", c[i * SIZE + j]);
        }
        printf("\n");
    }
}

Matrix C:
334 360 308 299 171 258 178 298 322 305 308 409 250 261 261 334 
357 444 345 321 220 323 268 292 403 355 344 375 184 219 276 349 
370 392 395 416 282 323 321 329 407 379 354 455 239 320 334 340 
372 405 279 308 215 252 304 288 338 392 340 419 273 239 300 311 
483 538 352 447 250 396 438 377 425 516 426 585 333 244 429 374 
446 460 425 439 286 339 341 361 513 422 456 517 273 363 389 417 
376 461 328 392 252 300 323 359 376 402 429 441 241 295 245 333 
340 461 362 403 272 302 344 235 401 377 409 446 236 317 263 385 
256 299 277 350 209 169 219 212 271 278 291 356 188 294 183 288 
483 508 449 425 261 399 401 444 465 498 463 549 247 270 368 390 
332 360 215 254 194 268 226 251 313 240 300 339 242 215 213 266 
341 434 307 390 240 269 314 353 363 387 412 492 267 302 284 333 
296 352 223 328 188 212 238 304 269 280 296 348 194 203 216 226 
404 469 336 430 234 342 306 387 435 389 422 525 229 315 294 369 
489 578 405 476 305 367 357 430 463 452 456 561 324 324 313 425 
328 411 303 353